# Programming Assignment 2

In this assignment, you will:

1. Practice writing code to train and evaluate models using both the two-way holdout method, and an evaluation approach appropriate for models with hyperparameters that uses k-fold cross validation plus a test set.

2. Practice writing code to optimize a machine learning model. In particular, we will use gradient descent to optimize a logistic regression model.

3. **574 Only**: Perform optimization with a different algorithm (Newton-Raphson)

# Resources you can use to complete this assignment (a COMPLETE list)

**NOTE: You ARE allowed to use Google to find things that fit this list (i.e. it is often easy to google something like "plotly draw line graph" to find the right part of the plotly documentation).**

- Anything linked to in this article
- Anything linked to from the course web page
- Any materials from another online course taught at a university (**if you use this, you MUST provide a link to the exact document used**)
- Anything posted by Kenny, Navid, or Yincheng on Piazza



# Grading

For grading of code in Part 2, we will execute the submitted notebook as follows:

```shell
jupyter nbconvert --to python Assignment2-Student.ipynb
python Assignment2-Student.py
```

The PDF that comes along with this document has other details on the points awarded for each part.

As such, you will submit, one member of your group will subit as a zip file on UBLearns, a ```.zip``` file that contains 4 things:
- Your completed jupyter notebook.
- Your written report, answering all questions asked here (and copied in the assignment PDF)
- `part_1.1_results.csv`
- `part_1.4_results.csv`


# Part 1 - Predicting Review Scores on Pitchfork

For Part 1, we will be using data from [this paper](https://ojs.aaai.org/index.php/ICWSM/article/view/7355). The data is a collection of reviews from [Pitchfork](https://pitchfork.com/), a site that provides expert reviews of music album. The authors of this paper have also combined the data with a set of features from [Spotify’s API](https://developer.spotify.com/documentation/web-api/) that provide insight into the music itself, e.g. the "acousticness" of the song.  We will tackle a regression problem here, trying to predict the score of a review from several of the other columns in the dataset.

## Part 1.1 - Feature Engineering with Feature Subsets

In the first subsection of Part 1, we’re going to rely on our old friend linear regression. We’re going to look at how running linear regression with various subsets of our features impacts our ability to predict score.

In Part 1.1, your task is to write code below that trains a separate linear regression model for a number of different feature subsets.  Specifically:

- The list `feature_sets` below is a list of lists; each sublist is a different subset of features to build a model with. 
- All models should be trained on the dataset `part1_train.csv`. 
- For each of these trained models, you should evaluate the model’s predictions on the training dataset, as well as the provided test set, called `part1_test.csv`. The evaluation metric we will use is **root mean squared error**.  

Write out the result to a file called `part_1.1_results.csv` and submit this along with your assignment. The file should have the following columns:
- `feature_set` - a column describing the features of the model used. For feature sets with multiple features, combine them using an underscore (you can do this with the code `"_".join(feature_set)`)
- `training_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the training data
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data

In addition, please answer the following questions:
- **1.1.1** Which model had the best RMSE on the *training data*? 
- **1.1.2** Which model had the best RMSE on the *test data*? 
- **1.1.3** Which feature do you believe was the most important one? Why? *(Note: There is more than one perfectly acceptable way to answer this question)*
- **1.1.4** What can we say about the utility of the Spotify features based on these results?

In [ ]:
feature_sets = [['artist'],
 ['reviewauthor'],
 ['releaseyear'],
 ['recordlabel'],
 ['genre'],
 ['danceability'],
 ['energy'],
 ['key'],
 ['loudness'],
 ['speechiness'],
 ['acousticness'],
 ['instrumentalness'],
 ['liveness'],
 ['valence'],
 ['tempo'],
 ['danceability','energy','key','loudness','speechiness','acousticness',
  'instrumentalness','liveness','valence','tempo'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre', 'danceability', 
  'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
  'liveness', 'valence', 'tempo']]


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Convenience things for you, note that releaseyear is continuous but is not a Spotify API variable
CONTINUOUS_FEATURES = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']
CATEGORICAL_FEATURES = ['artist', 'reviewauthor', 'recordlabel', 'genre']

# Read in the data
training_data = pd.read_csv("part1_train.csv")
test_data = pd.read_csv("part1_test.csv")

def onehot_encode_var(train, test, varname):
    # This function should take in a variable name in part3_data and return a onehot encoded matrix for that variable
    
    # Here's a starting point!
    encoder = OneHotEncoder(drop = "first", handle_unknown= 'ignore')

    # TODO: Replace
    full = pd.concat((train, test))
    encoder = encoder.fit(full[varname])

    train_encoded = encoder.transform(train[varname]).toarray()
    test_encoded = encoder.transform(test[varname]).toarray()
    
    # return the onehot encoded variable
    return train_encoded, test_encoded, encoder.get_feature_names_out()

training_result_data = []
test_result_data = []

def a(data, data2, result_data_set):
  y_train = np.log(data.score + 1) 
  y_test = np.log(data2.score + 1)

  for feature_set in feature_sets:
      # Write your code for Part 1.1 here!
      categorical_tmp = []
      continuous_tmp = []
      for x in feature_set:
          if x in CATEGORICAL_FEATURES:
            categorical_tmp.append(x)
          elif x in CONTINUOUS_FEATURES:
            continuous_tmp.append(x)
      
      if not categorical_tmp and continuous_tmp:
        training_continuous_rescaled_X = StandardScaler().fit_transform(data[continuous_tmp].values)
        test_continuous_rescaled_X = StandardScaler().fit_transform(data2[continuous_tmp].values)

        X_train = np.hstack([np.matrix(training_continuous_rescaled_X)])
        X_test = np.hstack([np.matrix(test_continuous_rescaled_X)])
        
      
      elif not continuous_tmp and categorical_tmp:
        train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(data, data2, 
                                                                                      categorical_tmp)
        X_train = np.hstack([train_onehot])
        X_test = np.hstack([test_onehot])
        
      
      elif continuous_tmp and categorical_tmp:
        training_continuous_rescaled_X = StandardScaler().fit_transform(data[continuous_tmp].values)
        test_continuous_rescaled_X = StandardScaler().fit_transform(data2[continuous_tmp].values)

        train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(data, data2, 
                                                                                      categorical_tmp)
        
        X_train = np.hstack([np.matrix(training_continuous_rescaled_X), 
                       train_onehot])
        
        X_test = np.hstack([np.matrix(test_continuous_rescaled_X), 
                       test_onehot])
       
      lmodel = LinearRegression()
      lmodel.fit(X_train,y_train)
      

      RMSE = np.sqrt(((lmodel.predict(X_test)-y_test)**2).sum()/len(y_test))
     
      result_data_set.append(RMSE)
      pass

tmp = feature_sets
csv_featuresets = []

for x in tmp:
    csv_featuresets.append("_".join(x))

a(training_data, test_data, test_result_data)
a(training_data, training_data, training_result_data)
df = pd.DataFrame({'feature_set': csv_featuresets,
                   'training_rmse': training_result_data,
                   'test_rmse' : test_result_data})
df.to_csv("part_1.1_results.csv", index = False)

## Part 1.2 - Feature Engineering with the LASSO

In Part 1.2, your task is to write code below that trains an L1-regularized linear regression model, with an expanded feature set.  Specifically:

1. Begin with the final feature set listed in `feature_sets` (i.e. your feature set, to begin this section, is `feature_sets[-1]`.
2. One-hot encode your categorical variables, setting `drop=if_binary` and `sparse=False` in the function arguments. 
3. Scale all of your continuous features using the `StandardScaler`.
4. Train an L1-regularized linear regression model using these features on the dataset `part1_train.csv`. You should use the [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) class in `sklearn`, it will do the cross-validation necessary to select the appropriate value for the regularizer for you!  Use 10-fold cross-validation to perform model selection (set the `LassoCV` parmaeter `cv` to 10), and set the `random_state` to 1. Do not change any of the other parameters to `LassoCV` (i.e. leave them at their defaults).
5. Identify the best `alpha` value (the regularizer term, according to `sklearn`. In class, we refer to this as $\lambda$!) in terms of average mean squared error according to the cross-validation.
6. Finally, train a [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) model on the entire training dataset (`part1_train.csv`). You will use this to report the root mean squared error on the test set (Question 1.2.4 below), and use it in Part 1.3 below as well.

**Hint: The proceedure outlined above is very similar to ones we have discussed in class and shown how to do using `Pipeline`s.** 


In [ ]:
# Write your code for Part 1.2 here
from sklearn.linear_model import LassoCV, Lasso


#거꾸로 돌면서 똑같이하되, RMSE젤 적은거 찾고 그거의 알파값을 구하면될듯
# Do the CV to find alpha


CONTINUOUS_FEATURES = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']
CATEGORICAL_FEATURES = ['artist', 'reviewauthor', 'recordlabel', 'genre']


def onehot_encode_var(train, test, varname):
    encoder = OneHotEncoder(drop = "if_binary", sparse= 'False')
    full = pd.concat((train, test))
    encoder = encoder.fit(full[varname])
    train_encoded = encoder.transform(train[varname]).toarray()
    test_encoded = encoder.transform(test[varname]).toarray()
    return train_encoded, test_encoded, encoder.get_feature_names_out()

training_result_data = []
test_result_data = []

alphas = []
rmses = []
def a(data, data2, result_data_set):
    y_train = np.log(data.score + 1) 
    y_test = np.log(data2.score + 1)
    for feature_set in reversed(feature_sets):
        categorical_tmp = []
        continuous_tmp = []
        for x in feature_set:
            if x in CATEGORICAL_FEATURES:
                categorical_tmp.append(x)
            elif x in CONTINUOUS_FEATURES:
                continuous_tmp.append(x)
        if not categorical_tmp and continuous_tmp:
            training_continuous_rescaled_X = StandardScaler().fit_transform(data[continuous_tmp].values)
            test_continuous_rescaled_X = StandardScaler().fit_transform(data2[continuous_tmp].values)
            X_train = np.hstack([np.matrix(training_continuous_rescaled_X)])
            X_test = np.hstack([np.matrix(test_continuous_rescaled_X)])
        
        elif not continuous_tmp and categorical_tmp:
            train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(data, data2,categorical_tmp)
            X_train = np.hstack([train_onehot])
            X_test = np.hstack([test_onehot])
        elif continuous_tmp and categorical_tmp:
            training_continuous_rescaled_X = StandardScaler().fit_transform(data[continuous_tmp].values)
            test_continuous_rescaled_X = StandardScaler().fit_transform(data2[continuous_tmp].values)
            train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(data, data2, categorical_tmp)
            X_train = np.hstack([np.matrix(training_continuous_rescaled_X), train_onehot])
            X_test = np.hstack([np.matrix(test_continuous_rescaled_X), test_onehot])
        lmodel = LassoCV(cv=10,random_state=1)
        lmodel.fit(X_train,y_train)
        
        alphas.append(lmodel.alpha_)
        
        RMSE = np.sqrt(((lmodel.predict(X_test)-y_test)**2).sum()/len(y_test))
        rmses.append(RMSE)
        
    


a(training_data, test_data, test_result_data)
min_RMSE = min(rmses)
min_RMSE_idx = rmses.index(min_RMSE)
optimal_alpha = alphas[min_RMSE_idx]



# Retrain the model
def b(data, data2, optimal_alpha):
    y_train = np.log(data.score + 1) 
    y_test = np.log(data2.score + 1)
    
    training_continuous_rescaled_X = StandardScaler().fit_transform(data[CONTINUOUS_FEATURES].values)
    test_continuous_rescaled_X = StandardScaler().fit_transform(data2[CONTINUOUS_FEATURES].values)
    train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(data, data2, CATEGORICAL_FEATURES)
    X_train = np.hstack([np.matrix(training_continuous_rescaled_X), train_onehot])
    X_test = np.hstack([np.matrix(test_continuous_rescaled_X), test_onehot])
    
    lmodel = Lasso(alpha = optimal_alpha)
    lmodel.fit(X_train,y_train)
      
    RMSE = np.sqrt(((lmodel.predict(X_test)-y_test)**2).sum()/len(y_test))

    return RMSE

print(b(training_data, training_data, optimal_alpha))
print(b(training_data, test_data, optimal_alpha))


0.17069160320998453
0.17069160320998453


# Part 1.3 - Interpreting Model Coefficents

In this section we will interpret the coefficients from the final model you trained on all of the training data.

- **1.3.1** - How many non-zero coefficients are in this final model?
- **1.3.2** - What percentage of the coefficients are non-zero in this final model?
- **1.3.3** - Who were the three most critical review authors, as estimated by the model? How do you know?
- **1.3.4** - Who were the three artists that reviewers tended to like the most?  How do you know?
- **1.3.5** - What genre did Pitchfork reviewers tend to like the most? Which genre did they like the least?


Now, answer the following questions:
- **1.2.1** - How many total features are introduced by Step 2 above? Provide both the number and an explanation of how you got to this number.
- **1.2.2** - What was the best `alpha` value according to your cross-validation results?
- **1.2.3** - What was the **average RMSE** of the model with this `alpha` value on the k-fold cross validation on the *training* data?
- **1.2.4** - What was the **RMSE** of the model with this `alpha` value on the k-fold cross validation on the *test* data?


# Part 1.4 - "Manual" Cross-Validation + Holdout for Model Selection and Evaluation

We will finally use cross validation for both algorithm and model selection, with a hold-out test set for a final evaluation. We will use **5-fold cross validation** to identify the best parameters and hyperparameters for a set of models. We will then take our final models and use a final hold-out test set (the same one as above) to estimate the generalization error of the models.

Specifically, your task is first to write code that trains and evaluates the following models, one for each of the specified hyper parameters sets:

- `Decision Tree regression` - All combinations of a `max_depth` of 5, 10, or 20, and a `criterion` of `"squared error"` or `"absolute error"`. Use the [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor).
- Ridge regression - Use the following choices of L2 penalty: $[10^{-5}, 10^{-4}, ..., 10^4, 10^5]$. In Python, you can create a list of these numbers using `np.logspace(-5, 5, 11)`. Use the [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) class from sklearn to train a Ridge Regression model. The parameters you need to pass when constructing the Ridge model are `alpha`, which lets you specify what you want the L2 penalty to be, and `random_state=0` to avoid randomness.
- kNN regression - Values of `n_neighbors` of 1, 5, 10, and 15. Use the [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) class.

Additional notes:
1. All models should use the feature sets described in Part 1.3 (the same ...one-hot encoded... categorical variables, and the scaled continuous variables)
2. As opposed to using the `KFold` class from `sklearn` like we did in class, we have instead provided you with pre-existing data sets; you should therefore use the pre-split data in the provided CSV files ``1.2_fold0.csv ... 1.2_fold4.csv``

**What to submit**:

1. Write out the result to a file called `part_1.4_results.csv` and submit this along with your assignment. The file should have the following columns:
- `model_name` - The name of the model, one of `DTR` (Decision Tree Regression), `Ridge`, or `KNN`.
- `hyperparameter_setting` - a column describing the hyperparameters of the model. For models with multiple hyperparameters, combine them using an underscore (you can do this with the code `"_".join(hyperparameters)`).
- `mean_training_rmse` - a column that gives the mean RMSE on the k-fold training data. You should take the average of the model’s errors on the different folds, using root mean squared error again as your evaluation metric.
- `sd_training_rmse` - a column that gives the standard deviation RMSE on the k-fold training data.
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data


2. Answer the following questions:
- **1.4.1** Report, for each model, the hyper parameter setting that resulted in the best performance
- **1.4.2** Which model performed the best overall on the cross-validation? 
- **1.4.3** Which model performed the best overall on the final test set? 
- **1.4.4** With respect to your answer for 1.4.3, why do you think that might be? (*Note: there is more than one correct way to answer this question*)
- **1.4.5** Which model/hyperparameter setting had the highest standard deviation across the different folds of the cross validation?
- **1.4.6** With respect to your answer for 1.4.6, why do you think that might be? (*Note: there is more than one correct way to answer this question*)

In [ ]:
feature_sets = [['artist'],
 ['reviewauthor'],
 ['releaseyear'],
 ['recordlabel'],
 ['genre'],
 ['danceability'],
 ['energy'],
 ['key'],
 ['loudness'],
 ['speechiness'],
 ['acousticness'],
 ['instrumentalness'],
 ['liveness'],
 ['valence'],
 ['tempo'],
 ['danceability','energy','key','loudness','speechiness','acousticness',
  'instrumentalness','liveness','valence','tempo'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre', 'danceability', 
  'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
  'liveness', 'valence', 'tempo']]

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor

# Convenience things for you, note that releaseyear is continuous but is not a Spotify API variable
CONTINUOUS_FEATURES = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']
CATEGORICAL_FEATURES = ['artist', 'reviewauthor', 'recordlabel', 'genre']

# Read in the data
fold0 = pd.read_csv("1.2_fold0.csv")
fold1 = pd.read_csv("1.2_fold1.csv")
fold2 = pd.read_csv("1.2_fold2.csv")
fold3 = pd.read_csv("1.2_fold3.csv")
fold4 = pd.read_csv("1.2_fold4.csv")
training_data = pd.read_csv("part1_train.csv")
test_data = pd.read_csv("part1_test.csv")

def onehot_encode_var(train, test, varname):
    # This function should take in a variable name in part3_data and return a onehot encoded matrix for that variable
    
    # Here's a starting point!
    encoder = OneHotEncoder(drop = "if_binary", sparse= 'False', handle_unknown='error')

    # TODO: Replace
    full = pd.concat((train, test))
    encoder = encoder.fit(full[varname])

    train_encoded = encoder.transform(train[varname]).toarray()
    test_encoded = encoder.transform(test[varname]).toarray()
    
    # return the onehot encoded variable
    return train_encoded, test_encoded, encoder.get_feature_names_out()

training_result_data = []
test_result_data = []

In [ ]:
#Decision Tree regression - Training

DTR_Training_Result = []

def part1_4(data_set):
  for i in range(len(data_set)):
    tmp_result = []

    validation_data = data_set[i]
    kfolds_tmp = []

    for j in range(len(data_set)):
      if j != i:
        kfolds_tmp.append(data_set[j])

    tmp_training_set = pd.concat(kfolds_tmp) 

    y_train = np.log(tmp_training_set.score + 1) 
    y_test = np.log(validation_data.score + 1)

    for feature_set in reversed(feature_sets):
        categorical_tmp = []
        continuous_tmp = []
        for x in feature_set:
            if x in CATEGORICAL_FEATURES:
              categorical_tmp.append(x)
            elif x in CONTINUOUS_FEATURES:
              continuous_tmp.append(x)
        
        if not categorical_tmp and continuous_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(tmp_training_set[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(validation_data[continuous_tmp].values)

          X_train = np.hstack([np.matrix(training_continuous_rescaled_X)])
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X)])
          
        elif not continuous_tmp and categorical_tmp:
          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(tmp_training_set, validation_data, 
                                                                                        categorical_tmp)
          X_train = np.hstack([train_onehot])
          X_test = np.hstack([test_onehot])
          
        elif continuous_tmp and categorical_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(tmp_training_set[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(validation_data[continuous_tmp].values)

          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(tmp_training_set, validation_data, 
                                                                                        categorical_tmp)    
          X_train = np.hstack([np.matrix(training_continuous_rescaled_X), 
                        train_onehot])        
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X), 
                        test_onehot])
        
        DTR1 = DecisionTreeRegressor(max_depth=5, criterion="squared_error") 
        DTR2 = DecisionTreeRegressor(max_depth=10, criterion="squared_error")
        DTR3 = DecisionTreeRegressor(max_depth=20, criterion="squared_error")
      
        DTR1.fit(X_train, y_train)
        DTR2.fit(X_train, y_train)
        DTR3.fit(X_train, y_train)
        
        DTR1_RMSE = np.sqrt(((DTR1.predict(X_test)-y_test)**2).sum()/len(y_test))
        DTR2_RMSE = np.sqrt(((DTR2.predict(X_test)-y_test)**2).sum()/len(y_test))
        DTR3_RMSE = np.sqrt(((DTR3.predict(X_test)-y_test)**2).sum()/len(y_test))
        
        tmp = [DTR1_RMSE, DTR2_RMSE, DTR3_RMSE]
        
        tmp_result.append(tmp)

    DTR_Training_Result.append(tmp_result)

part1_4([fold0, fold1, fold2, fold3, fold4])



In [ ]:
# Extract the data to csv file to calculate mean and SD

fRow = DTR_Training_Result[0]
df = pd.DataFrame([fRow[0]])

for x in range(len(DTR_Training_Result)):
  for y in DTR_Training_Result[x]:
    df.loc[len(df)] = y

df.to_csv("part_1.4_DTR_Training_results.csv", index = False)

In [ ]:
#Decision Tree regression - Testing
DTR_Test_Result = []

def part1_4(data, data2):
    y_train = np.log(data.score + 1) 
    y_test = np.log(data2.score + 1)

    for feature_set in reversed(feature_sets):
        categorical_tmp = []
        continuous_tmp = []
        for x in feature_set:
            if x in CATEGORICAL_FEATURES:
              categorical_tmp.append(x)
            elif x in CONTINUOUS_FEATURES:
              continuous_tmp.append(x)
        
        if not categorical_tmp and continuous_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(data[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(data2[continuous_tmp].values)

          X_train = np.hstack([np.matrix(training_continuous_rescaled_X)])
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X)])
          
        elif not continuous_tmp and categorical_tmp:
          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(data, data2, 
                                                                                        categorical_tmp)
          X_train = np.hstack([train_onehot])
          X_test = np.hstack([test_onehot])
          
        elif continuous_tmp and categorical_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(data[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(data2[continuous_tmp].values)

          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(data, data2, 
                                                                                        categorical_tmp)    
          X_train = np.hstack([np.matrix(training_continuous_rescaled_X), 
                        train_onehot])        
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X), 
                        test_onehot])
        
        DTR1 = DecisionTreeRegressor(max_depth=5, criterion="squared_error") #BEST ONE
        DTR2 = DecisionTreeRegressor(max_depth=10, criterion="squared_error")
        DTR3 = DecisionTreeRegressor(max_depth=20, criterion="squared_error")
      
        DTR1.fit(X_train, y_train)
        DTR2.fit(X_train, y_train)
        DTR3.fit(X_train, y_train)
        
        DTR1_RMSE = np.sqrt(((DTR1.predict(X_test)-y_test)**2).sum()/len(y_test))
        DTR2_RMSE = np.sqrt(((DTR2.predict(X_test)-y_test)**2).sum()/len(y_test))
        DTR3_RMSE = np.sqrt(((DTR3.predict(X_test)-y_test)**2).sum()/len(y_test))

        tmp = [DTR1_RMSE, DTR2_RMSE, DTR3_RMSE]
        
        DTR_Test_Result.append(tmp)
        pass
      
tmp_train = pd.concat([fold0, fold1, fold2, fold3, fold4])
part1_4(tmp_train, test_data)

In [ ]:
# 1.4.3 Which model performed the best overall on the final test set?
avg_list = []
for i in range(len(DTR_Test_Result[0])):
  sum = 0
  for j in DTR_Test_Result:
     sum += j[i]
  avg_list.append(sum/len(DTR_Test_Result))

total = 0
for x in avg_list:
  total += x

print(total/len(avg_list))


0.19996162902161838


In [ ]:
# Ridge regression - Training
Ridge_Training_result = []

def part1_4(data_set):
  for i in range(len(data_set)):
    validation_data = data_set[i]
    kfolds_tmp = []

    tmp_result2 = []
    for j in range(len(data_set)):
      if j != i:
        kfolds_tmp.append(data_set[j])

    tmp_training_set = pd.concat(kfolds_tmp) 

    y_train = np.log(tmp_training_set.score + 1) 
    y_test = np.log(validation_data.score + 1)

    for feature_set in reversed(feature_sets):
        categorical_tmp = []
        continuous_tmp = []
        for x in feature_set:
            if x in CATEGORICAL_FEATURES:
              categorical_tmp.append(x)
            elif x in CONTINUOUS_FEATURES:
              continuous_tmp.append(x)
        
        if not categorical_tmp and continuous_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(tmp_training_set[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(validation_data[continuous_tmp].values)

          X_train = np.hstack([np.matrix(training_continuous_rescaled_X)])
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X)])
          
        elif not continuous_tmp and categorical_tmp:
          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(tmp_training_set, validation_data, 
                                                                                        categorical_tmp)
          X_train = np.hstack([train_onehot])
          X_test = np.hstack([test_onehot])
          
        elif continuous_tmp and categorical_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(tmp_training_set[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(validation_data[continuous_tmp].values)

          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(tmp_training_set, validation_data, 
                                                                                        categorical_tmp)    
          X_train = np.hstack([np.matrix(training_continuous_rescaled_X), 
                        train_onehot])        
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X), 
                        test_onehot])
        
        alphas = np.logspace(-5, 5, 11)
        tmp = []                                                                                                                                                                                                       
        for alpha in alphas:
          ridge = Ridge(alpha=alpha, random_state=0)
          ridge.fit(X_train, y_train)
          ridge_RMSE = np.sqrt(((ridge.predict(X_test)-y_test)**2).sum()/len(y_test))
          tmp.append(ridge_RMSE)

        tmp_result2.append(tmp)

    Ridge_Training_result.append(tmp_result2)

part1_4([fold0, fold1, fold2, fold3, fold4])


In [ ]:
# Extract the data to csv file to calculate mean and SD
avg_set = []

fRow = Ridge_Training_result[0]
df = pd.DataFrame([fRow[0]])

for x in range(len(Ridge_Training_result)):
  for y in Ridge_Training_result[x]:
    df.loc[len(df)] = y

df.to_csv("part_1.4_Ridge_Training_results.csv", index = False)

In [ ]:
# Ridge regression - Testing
Ridge_Test_result = []

def part1_4(data, data2):
    y_train = np.log(data.score + 1) 
    y_test = np.log(data2.score + 1)

    for feature_set in reversed(feature_sets):
        categorical_tmp = []
        continuous_tmp = []
        for x in feature_set:
            if x in CATEGORICAL_FEATURES:
              categorical_tmp.append(x)
            elif x in CONTINUOUS_FEATURES:
              continuous_tmp.append(x)
        
        if not categorical_tmp and continuous_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(data[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(data2[continuous_tmp].values)

          X_train = np.hstack([np.matrix(training_continuous_rescaled_X)])
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X)])
          
        elif not continuous_tmp and categorical_tmp:
          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(data, data2, 
                                                                                        categorical_tmp)
          X_train = np.hstack([train_onehot])
          X_test = np.hstack([test_onehot])
          
        elif continuous_tmp and categorical_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(data[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(data2[continuous_tmp].values)

          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(data, data2, 
                                                                                        categorical_tmp)    
          X_train = np.hstack([np.matrix(training_continuous_rescaled_X), 
                        train_onehot])        
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X), 
                        test_onehot])
        
        alphas = np.logspace(-5, 5, 11)
        tmp = []
        for alpha in alphas:
          ridge = Ridge(alpha=alpha, random_state=0)
          ridge.fit(X_train, y_train)
          ridge_RMSE = np.sqrt(((ridge.predict(X_test)-y_test)**2).sum()/len(y_test))
          tmp.append(ridge_RMSE)

        Ridge_Test_result.append(tmp)
      
tmp_train = pd.concat([fold0, fold1, fold2, fold3, fold4])
part1_4(tmp_train, test_data)

In [ ]:
# 1.4.3 Which model performed the best overall on the final test set?

avg_list = []
for i in range(len(Ridge_Test_result[0])):
  sum = 0
  for j in Ridge_Test_result:
     sum += j[i]
  avg_list.append(sum/len(Ridge_Test_result))

total = 0
for x in avg_list:
  total += x

print(total/len(avg_list))


0.1924572535445589


In [ ]:
ridge_test_rmse = []

for i in range(len(Ridge_Test_result[0])):
  sum = 0
  for j in Ridge_Test_result:
    sum += j[i]
  ridge_test_rmse.append(sum/len(Ridge_Test_result))


In [ ]:
#KNR - Training

KNR_Training_result = []

def part1_4(data_set):
  for i in range(len(data_set)):
    validation_data = data_set[i]
    kfolds_tmp = []
    tmp_result3 = []
    
    for j in range(len(data_set)):
      if j != i:
        kfolds_tmp.append(data_set[j])

    tmp_training_set = pd.concat(kfolds_tmp) 

    y_train = np.log(tmp_training_set.score + 1) 
    y_test = np.log(validation_data.score + 1)

    for feature_set in reversed(feature_sets):
        categorical_tmp = []
        continuous_tmp = []
        for x in feature_set:
            if x in CATEGORICAL_FEATURES:
              categorical_tmp.append(x)
            elif x in CONTINUOUS_FEATURES:
              continuous_tmp.append(x)
        
        if not categorical_tmp and continuous_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(tmp_training_set[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(validation_data[continuous_tmp].values)

          X_train = np.hstack([np.matrix(training_continuous_rescaled_X)])
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X)])
          
        elif not continuous_tmp and categorical_tmp:
          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(tmp_training_set, validation_data, 
                                                                                        categorical_tmp)
          X_train = np.hstack([train_onehot])
          X_test = np.hstack([test_onehot])
          
        elif continuous_tmp and categorical_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(tmp_training_set[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(validation_data[continuous_tmp].values)

          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(tmp_training_set, validation_data, 
                                                                                        categorical_tmp)    
          X_train = np.hstack([np.matrix(training_continuous_rescaled_X), 
                        train_onehot])        
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X), 
                        test_onehot])
        
        tmp = []
        KNR1 = KNeighborsRegressor(n_neighbors=1)
        KNR2 = KNeighborsRegressor(n_neighbors=5)
        KNR3 = KNeighborsRegressor(n_neighbors=10)
        KNR4 = KNeighborsRegressor(n_neighbors=15)

        KNR1.fit(X_train, y_train)
        KNR2.fit(X_train, y_train)
        KNR3.fit(X_train, y_train)
        KNR4.fit(X_train, y_train)

        KNR1_RMSE = np.sqrt(((KNR1.predict(X_test)-y_test)**2).sum()/len(y_test))
        KNR2_RMSE = np.sqrt(((KNR2.predict(X_test)-y_test)**2).sum()/len(y_test))
        KNR3_RMSE = np.sqrt(((KNR3.predict(X_test)-y_test)**2).sum()/len(y_test))
        KNR4_RMSE = np.sqrt(((KNR4.predict(X_test)-y_test)**2).sum()/len(y_test))

        tmp = [KNR1_RMSE, KNR2_RMSE, KNR3_RMSE, KNR4_RMSE]
       
        tmp_result3.append(tmp)

    KNR_Training_result.append(tmp_result3)
part1_4([fold0, fold1, fold2, fold3, fold4])

In [ ]:
# Extract the data to csv file to calculate mean and SD

fRow = KNR_Training_result[0]
df = pd.DataFrame([fRow[0]])

for x in range(len(KNR_Training_result)):
  for y in KNR_Training_result[x]:
    df.loc[len(df)] = y

df.to_csv("part_1.4_KNR_Training_results.csv", index = False)

In [ ]:
#KNR - Testing
KNR_Test_result = []

def part1_4(data, data2):
    y_train = np.log(data.score + 1) 
    y_test = np.log(data2.score + 1)

    for feature_set in reversed(feature_sets):
        categorical_tmp = []
        continuous_tmp = []
        for x in feature_set:
            if x in CATEGORICAL_FEATURES:
              categorical_tmp.append(x)
            elif x in CONTINUOUS_FEATURES:  
              continuous_tmp.append(x)
        
        if not categorical_tmp and continuous_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(data[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(data2[continuous_tmp].values)

          X_train = np.hstack([np.matrix(training_continuous_rescaled_X)])
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X)])
          
        elif not continuous_tmp and categorical_tmp:
          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(data, data2, 
                                                                                        categorical_tmp)
          X_train = np.hstack([train_onehot])
          X_test = np.hstack([test_onehot])
          
        elif continuous_tmp and categorical_tmp:
          training_continuous_rescaled_X = StandardScaler().fit_transform(data[continuous_tmp].values)
          test_continuous_rescaled_X = StandardScaler().fit_transform(data2[continuous_tmp].values)

          train_onehot, test_onehot, train_onehot_encoded_subreddit_feature_set = onehot_encode_var(data, data2, 
                                                                                        categorical_tmp)    
          X_train = np.hstack([np.matrix(training_continuous_rescaled_X), 
                        train_onehot])        
          X_test = np.hstack([np.matrix(test_continuous_rescaled_X), 
                        test_onehot])
        
        
        tmp = []
        KNR1 = KNeighborsRegressor(n_neighbors=1)
        KNR2 = KNeighborsRegressor(n_neighbors=5)
        KNR3 = KNeighborsRegressor(n_neighbors=10)
        KNR4 = KNeighborsRegressor(n_neighbors=15)

        KNR1.fit(X_train, y_train)
        KNR2.fit(X_train, y_train)
        KNR3.fit(X_train, y_train)
        KNR4.fit(X_train, y_train)

        KNR1_RMSE = np.sqrt(((KNR1.predict(X_test)-y_test)**2).sum()/len(y_test))
        KNR2_RMSE = np.sqrt(((KNR2.predict(X_test)-y_test)**2).sum()/len(y_test))
        KNR3_RMSE = np.sqrt(((KNR3.predict(X_test)-y_test)**2).sum()/len(y_test))
        KNR4_RMSE = np.sqrt(((KNR4.predict(X_test)-y_test)**2).sum()/len(y_test))

        tmp = [KNR1_RMSE, KNR2_RMSE, KNR3_RMSE, KNR4_RMSE]
        
        KNR_Test_result.append(tmp)
      
tmp_train = pd.concat([fold0, fold1, fold2, fold3, fold4])
part1_4(tmp_train, test_data)

In [ ]:
# 1.4.3 Which model performed the best overall on the final test set?
avg_list = []
for i in range(len(KNR_Test_result[0])):
  sum = 0
  for j in KNR_Test_result:
    sum += j[i]
  
  avg_list.append(sum/len(KNR_Test_result))

total = 0
for x in avg_list:
  total += x

print(total/len(avg_list))

0.21962672946488204


In [ ]:
hyperparameter_setting = []
for i in range(-5, 6):
  setting = "alpha = 10^" + str(i)
  hyperparameter_setting.append(setting)

# Caculated using csv file that contains extracted data
mean_training_rmse = [0.1847779876247, 0.1847779760165, 0.1847778599668 ,0.1847767026362 ,0.1847654303466	,
                      0.1846758058301,	0.1844843225535,	0.1849255132430,	0.1856847011780,	0.1860895438290,	0.1863518392379]

# Caculated using csv file that contains extracted data                   
sd_training_rmse = [0.008080144,	0.008080154,	0.008080253, 0.008081239,	0.008090882, 0.008171447,	0.008415814,	0.008226803,	0.007936292,	0.007895813,	0.007888461]
df = pd.DataFrame({'model_name': "Ridge",
                   'hyperparameter_setting': hyperparameter_setting,
                   "mean_training_rmse" : mean_training_rmse,
                   "sd_training_rmse" : sd_training_rmse,
                   "test_rmse" : ridge_test_rmse})

df.to_csv("part_1.4_results.csv", index = False)

# Part 2

In class, we have shifted from regression to classification. Here, we're going to get a little practice in optimizing one of the classification models we saw in class - logistic regression. As a reminder...

The loss function of logistic regression (also known as the logistic-loss or log-loss) is given by:
\begin{equation}
  J({\bf w}) = \frac{1}{n}\sum_{i=1}^n \log{(1 + \exp{(-y_i{\bf w}^\top{\bf x}_i}))}
  \label{eqn:logloss}
\end{equation}

The gradient for this loss function, as derived in class, is:
\begin{equation}
  \nabla J({\bf w}) = -\frac{1}{n}\sum_{i=1}^n \frac{y_i}{1 + \exp{(y_i{\bf w}^\top{\bf x}_i)}}{\bf x}_i
  \label{eqn:loglossgradient}
\end{equation}


The Hessian for the loss function is given by:
\begin{equation}
  {\bf H}({\bf w}) = \frac{1}{n} \sum_{i=1}^n \frac{\exp{(y_i{\bf w}^\top{\bf x}_i)}}{(1 + \exp{(y_i{\bf w}^\top{\bf x}_i)})^2}{\bf x}_i{\bf x}_i^\top
  \label{eqn:loglosshessian}
\end{equation}

## Part 2.1 - Logistic Regression with Gradient Descent

In Part 2.1 we will implement logistic regression with gradient descent. You need to finish implementing 3 functions:

1. `logistic_objective` - compute the logistic loss for the given data set (see equation above)
2. `logistic_gradient` - compute the gradient vector of logistic loss for the given data set (see equation above)
3. `run_gradient_descent` - run the gradient descent algorithm, given these two functions.

We have provided you with some simulation data to evaluate your method with. Part 2.1 will, however, largely be graded by evaluating your code on a slightly different dataset to ensure robustness. 

In addition, please submit answers to the following questions on your written report:

- **2.1.1** - How did you go about selecting a good step size, i.e. one that was not too big or too small? (*Note: There is more than one correct answer to this*)
- **2.1.2** - What is the condition under which we assume that the gradient descent algorithm has converged in the code here?
- **2.1.3** - What is a different convergence metric we could have used? (*Note: There is more than one correct answer to this*)

In [ ]:
def logistic_objective(w, X, y):

    # compute log-loss error (scalar) with respect
    # to w (vector) for the given data X and y                               
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = scalar
    
    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    error = 0
    return error

def logistic_gradient(w, X, y):

    # compute the gradient of the log-loss error (vector) with respect
    # to w (vector) for the given data X and y  
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = d length gradient vector (not a d x 1 matrix)

    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    gradient = 0
    return gradient

def run_gradient_descent(X,y):
    old_w = np.array([-1]*X.shape[1])
    # change this value! This is an unreasonable step size
    step_size = 10000
    new_w =old_w - 1
    
    while ((new_w-old_w)**2).sum() > .0000000001:
        #IMPLEMENT THIS!
        pass
    return new_w



In [ ]:
from scipy.stats import uniform, bernoulli
import functools
draw_binary = functools.partial(np.random.binomial,n=1)

## Simulated data to test your method
DATA_SIZE = 10000
x1 = bernoulli(.5).rvs(DATA_SIZE)
x2 = np.floor(uniform(18,60).rvs(DATA_SIZE))
true_w = [-9, 3.5, 0.2]
xb = true_w[0] + true_w[1]*x1 + true_w[2]*x2
p = 1/(1 + np.exp(-xb))
y = np.array([1 if draw_binary(p=v) else -1 for v in p])

In [ ]:
from sklearn.linear_model import LogisticRegression

# notice that logistic regression as implemented in sklearn does not get the exact results either!
# so you shouldn't worry if you're a bit off
X = np.hstack([np.ones((len(xb),1)), x1[:,np.newaxis], x2[:,np.newaxis]])
model = LogisticRegression(solver='liblinear', random_state=0,fit_intercept=False)
model.fit(X,y).coef_

array([[-8.58020492,  3.40771205,  0.19037032]])

In [ ]:
# this is how we will test your results
gd_result = run_gradient_descent(X,y)
# is your result relatively close to the truth?
np.abs(true_w-gd_result).sum()

## <span style="color:red"> 574 Only</span> Part 2.2 - Optimization with Newton-Raphson <span style="color:red"> 574 Only</span>

In Part 2.2, you are going to, instead of using gradient descent, use the Newton-Raphson method to optimize the same logistic regression model. To do so, you will need to 1) implement the `logistic_hessian` function to compute the Hessian matrix of logistic loss for the given data set, and 2) use `scipy`'s `optimize` function to perform the optimization, rather than writing a function by hand to do so.  

For Part 2.2, you will only need to implement these functions, we will test them using our own code. You can, however, perform the same kinds of tests that we proposed above to check your work! 

In [ ]:
def logistic_hessian(w, X, y):

    # compute the Hessian of the log-loss error (matrix) with respect
    # to w (vector) for the given data X and y                               
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # Hessian = d x d matrix
    
    
    if len(w.shape) == 1:
        w = w[:,np.newaxis]

    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    hessian = 0
    return hessian

In [ ]:
from scipy.optimize import minimize

def run_newton_raphson(X,y):
    args = (X,y[:,np.newaxis])
    opts = {'maxiter' : 50}    # Preferred value.    
    w_init = np.zeros(X.shape[1])
    
    # note: this is *almost* what you need, you just need to figure out what arguments are necessary here!
    soln = minimize(PUT_THE_RIGHT_ARGUMENTS_IN_HERE!
                    args=args,
                    method='Newton-CG',
                    options=opts)

    w = np.transpose(np.array(soln.x))
    w = w[:,np.newaxis]
    return w


In [ ]:
run_newton_raphson(X,y)